# Exercise 1
## 1 Loading the Dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,train_test_split

In [33]:
#load dataset
digits = load_digits ()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

#data filtering 
num_1, num_2 = 3, 8
mask = np.logical_or(target == num_1, target == num_2)
data = data[mask]
target = target[mask]

#add column of 1's
data = np.hstack((data,np.ones((len(data),1))))

#relabel targets
target[target == num_1] = 1
target[target == num_2] = -1

### 1.1 Classification with sklearn

### 1.2 Optimization Methods